# Plagiarism Detection Model

Now that you've created training and test data, you are ready to define and train a model. Your goal in this notebook, will be to train a binary classification model that learns to label an answer file as either plagiarized or not, based on the features you provide the model.

This task will be broken down into a few discrete steps:

* Upload your data to S3.
* Define a binary classification model and a training script.
* Train your model and deploy it.
* Evaluate your deployed classifier and answer some questions about your approach.

To complete this notebook, you'll have to complete all given exercises and answer all the questions in this notebook.
> All your tasks will be clearly labeled **EXERCISE** and questions as **QUESTION**.

It will be up to you to explore different classification models and decide on a model that gives you the best performance for this dataset.

---

## Load Data to S3

In the last notebook, you should have created two files: a `training.csv` and `test.csv` file with the features and class labels for the given corpus of plagiarized/non-plagiarized text data. 

>The below cells load in some AWS SageMaker libraries and creates a default bucket. After creating this bucket, you can upload your locally stored data to S3.

Save your train and test `.csv` feature files, locally. To do this you can run the second notebook "2_Plagiarism_Feature_Engineering" in SageMaker or you can manually upload your files to this notebook using the upload icon in Jupyter Lab. Then you can upload local files to S3 by using `sagemaker_session.upload_data` and pointing directly to where the training data is saved.

In [1]:
import pandas as pd
import boto3
import sagemaker

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

## EXERCISE: Upload your training data to S3

Specify the `data_dir` where you've saved your `train.csv` file. Decide on a descriptive `prefix` that defines where your data will be uploaded in the default S3 bucket. Finally, create a pointer to your training data by calling `sagemaker_session.upload_data` and passing in the required parameters. It may help to look at the [Session documentation](https://sagemaker.readthedocs.io/en/stable/session.html#sagemaker.session.Session.upload_data) or previous SageMaker code examples.

You are expected to upload your entire directory. Later, the training script will only access the `train.csv` file.

In [3]:
# should be the name of directory you created to save your features data
data_dir = 'plagiarism_data'

# set prefix, a descriptive name for a directory  
prefix = 'plagiarism_analysis'

# upload all data to S3
data_s3 = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix = prefix)

In [4]:
data_s3

's3://sagemaker-us-east-1-980497197870/plagiarism_analysis'

### Test cell

Test that your data has been successfully uploaded. The below cell prints out the items in your S3 bucket and will throw an error if it is empty. You should see the contents of your `data_dir` and perhaps some checkpoints. If you see any other files listed, then you may have some old model files that you can delete via the S3 console (though, additional files shouldn't affect the performance of model developed in this notebook).

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# confirm that data is in S3 bucket
empty_check = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    print(obj.key)

assert len(empty_check) !=0, 'S3 bucket is empty.'
print('Test passed!')

counties/kmeans-2020-08-05-03-06-32-814/output/model.tar.gz
counties/kmeans-2020-08-05-19-42-17-609/output/model.tar.gz
counties/pca-2020-08-03-21-33-27-579/output/model.tar.gz
counties/pca-2020-08-05-02-18-45-709/output/model.tar.gz
counties/pca-2020-08-05-16-00-42-002/output/model.tar.gz
energy_consumption/output/energy-08-14-20-01-2020-08-14-23-24-51-796/output/model.tar.gz
energy_consumption/test/test.json
energy_consumption/train/train.json
fraud/linear-learner-2020-08-07-19-19-53-391/output/model.tar.gz
fraud/linear-learner-2020-08-07-19-51-40-003/output/model.tar.gz
fraud/linear-learner-2020-08-07-20-15-50-554/output/model.tar.gz
fraud/linear-learner-2020-08-07-20-35-34-354/output/model.tar.gz
fraud/linear-learner-2020-08-07-21-00-56-729/output/model.tar.gz
plagiarism_analysis/test.csv
plagiarism_analysis/train.csv
sagemaker-pytorch-2020-07-26-12-56-39-969/debug-output/training_job_end.ts
sagemaker-pytorch-2020-07-26-12-56-39-969/output/model.tar.gz
sagemaker-pytorch-2020-07-26-

---

# Modeling

Now that you've uploaded your training data, it's time to define and train a model!

The type of model you create is up to you. For a binary classification task, you can choose to go one of three routes:
* Use a built-in classification algorithm, like LinearLearner.
* Define a custom Scikit-learn classifier, a comparison of models can be found [here](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html).
* Define a custom PyTorch neural network classifier. 

It will be up to you to test out a variety of models and choose the best one. Your project will be graded on the accuracy of your final model. 
 
---

## EXERCISE: Complete a training script 

To implement a custom classifier, you'll need to complete a `train.py` script. You've been given the folders `source_sklearn` and `source_pytorch` which hold starting code for a custom Scikit-learn model and a PyTorch model, respectively. Each directory has a `train.py` training script. To complete this project **you only need to complete one of these scripts**; the script that is responsible for training your final model.

A typical training script:
* Loads training data from a specified directory
* Parses any training & model hyperparameters (ex. nodes in a neural network, training epochs, etc.)
* Instantiates a model of your design, with any specified hyperparams
* Trains that model 
* Finally, saves the model so that it can be hosted/deployed, later

### Defining and training a model
Much of the training script code is provided for you. Almost all of your work will be done in the `if __name__ == '__main__':` section. To complete a `train.py` file, you will:
1. Import any extra libraries you need
2. Define any additional model training hyperparameters using `parser.add_argument`
2. Define a model in the `if __name__ == '__main__':` section
3. Train the model in that same section

Below, you can use `!pygmentize` to display an existing `train.py` file. Read through the code; all of your tasks are marked with `TODO` comments. 

**Note: If you choose to create a custom PyTorch model, you will be responsible for defining the model in the `model.py` file,** and a `predict.py` file is provided. If you choose to use Scikit-learn, you only need a `train.py` file; you may import a classifier from the `sklearn` library.

In [10]:
# directory can be changed to: source_sklearn or source_pytorch
!pygmentize source_sklearn/train.py

from __future__ import print_function

import argparse
import os
import pandas as pd

from sklearn.neighbors import KNeighborsClassifier
from sklearn.externals import joblib

## TODO: Import any additional libraries you need to define a model


# Provided model load function
def model_fn(model_dir):
    """Load model from the model_dir. This is the same model that is saved
    in the main if statement.
    """
    print("Loading model.")
    
    # load using joblib
    model = joblib.load(os.path.join(model_dir, "model.joblib"))
    print("Done loading model.")
    
    return model


## TODO: Complete the main code
if __name__ == '__main__':
    
    # All of the model parameters and training parameters are sent as arguments
    # when this script is executed, during a training job
    
    # Here we set up an argument parser to easily access the parameters
    parser = argparse.ArgumentParser()

    # SageMaker parameters, like the directories for training data and saving models; se

### Provided code

If you read the code above, you can see that the starter code includes a few things:
* Model loading (`model_fn`) and saving code
* Getting SageMaker's default hyperparameters
* Loading the training data by name, `train.csv` and extracting the features and labels, `train_x`, and `train_y`

If you'd like to read more about model saving with [joblib for sklearn](https://scikit-learn.org/stable/modules/model_persistence.html) or with [torch.save](https://pytorch.org/tutorials/beginner/saving_loading_models.html), click on the provided links.

---
# Create an Estimator

When a custom model is constructed in SageMaker, an entry point must be specified. This is the Python file which will be executed when the model is trained; the `train.py` function you specified above. To run a custom training script in SageMaker, construct an estimator, and fill in the appropriate constructor arguments:

* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `source_sklearn` OR `source_pytorch`.
* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `train_sklearn` OR `train_pytorch`.
* **entry_point**: The path to the Python script SageMaker runs for training.
* **source_dir**: The path to the training script directory `train_sklearn` OR `train_pytorch`.
* **role**: Role ARN, which was specified, above.
* **train_instance_count**: The number of training instances (should be left at 1).
* **train_instance_type**: The type of SageMaker instance for training. Note: Because Scikit-learn does not natively support GPU training, Sagemaker Scikit-learn does not currently support training on GPU instance types.
* **sagemaker_session**: The session used to train on Sagemaker.
* **hyperparameters** (optional): A dictionary `{'name':value, ..}` passed to the train function as hyperparameters.

Note: For a PyTorch model, there is another optional argument **framework_version**, which you can set to the latest version of PyTorch, `1.0`.

## EXERCISE: Define a Scikit-learn or PyTorch estimator

To import your desired estimator, use one of the following lines:
```
from sagemaker.sklearn.estimator import SKLearn
```
```
from sagemaker.pytorch import PyTorch
```

In [41]:
# your import and estimator code, here
## I choose to use 0.20.0 is because the lastest version of scikit_learn deprecate the 'joblib' funtion 
# NNeighborsClassifer n = 5
from sagemaker.sklearn.estimator import SKLearn

model = SKLearn(entry_point = 'train.py',
                source_dir = 'source_sklearn',
                role = role,
                train_instance_count = 1,
                train_instance_type = 'ml.c4.xlarge',
                sagemaker_session = sagemaker_session,
                framework_version = '0.20.0')

This is not the latest supported version. If you would like to use version 0.23-1, please add framework_version=0.23-1 to your constructor.


In [49]:
# NNeighborsClassifer n = 3
model = SKLearn(entry_point = 'train.py',
                source_dir = 'source_sklearn',
                role = role,
                train_instance_count = 1,
                train_instance_type = 'ml.c4.xlarge',
                sagemaker_session = sagemaker_session,
                framework_version = '0.20.0',
                hyperparameters = {'n-neighbors': 3})

This is not the latest supported version. If you would like to use version 0.23-1, please add framework_version=0.23-1 to your constructor.


In [95]:
# Pytorch
from sagemaker.pytorch import PyTorch
model = PyTorch(entry_point = 'train.py',
                source_dir = 'source_pytorch',
                role = role,
                train_instance_count = 1,
                train_instance_type = 'ml.p2.xlarge',
                sagemaker_session = sagemaker_session,
                framework_version = '1.5.0',
                hyperparameters = {'epochs': 20,
                                   'input_features': 3,
                                   'hidden_dim': 30, 
                                   'output_dim': 1})

## EXERCISE: Train the estimator

Train your estimator on the training data stored in S3. This should create a training job that you can monitor in your SageMaker console.

In [50]:
%%time

# Train your estimator on S3 training data
model.fit({'train': data_s3})

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2020-08-20 03:33:35 Starting - Starting the training job...
2020-08-20 03:33:38 Starting - Launching requested ML instances.........
2020-08-20 03:35:20 Starting - Preparing the instances for training......
2020-08-20 03:36:20 Downloading - Downloading input data...
2020-08-20 03:36:50 Training - Downloading the training image..2020-08-20 03:37:14,246 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2020-08-20 03:37:14,248 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-08-20 03:37:14,259 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-08-20 03:37:14,537 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2020-08-20 03:37:14,538 sagemaker-containers INFO     Generating setup.cfg
2020-08-20 03:37:14,538 sagemaker-containers INFO     Generating MANIFEST.in
2020-08-20 03:37:14,538 sagemaker-containers INFO     Installing module with the followin

In [97]:
%%time

# Train your estimator on S3 training data
model.fit({'train': data_s3})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-08-21 02:22:55 Starting - Starting the training job...
2020-08-21 02:22:57 Starting - Launching requested ML instances......
2020-08-21 02:24:10 Starting - Preparing the instances for training............
2020-08-21 02:26:01 Downloading - Downloading input data...
2020-08-21 02:26:42 Training - Downloading the training image.........
2020-08-21 02:28:22 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-08-21 02:28:22,813 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-08-21 02:28:22,839 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-08-21 02:28:22,843 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-08-21 02:28:23,169 sagemaker-containers INFO     Module default_user_module_name does not provide a setup.py. 
Generating setup.py
2


2020-08-21 02:28:45 Uploading - Uploading generated training model
2020-08-21 02:28:45 Completed - Training job completed
Training seconds: 164
Billable seconds: 164
CPU times: user 778 ms, sys: 43.8 ms, total: 821 ms
Wall time: 6min 14s


## EXERCISE: Deploy the trained model

After training, deploy your model to create a `predictor`. If you're using a PyTorch model, you'll need to create a trained `PyTorchModel` that accepts the trained `<model>.model_data` as an input parameter and points to the provided `source_pytorch/predict.py` file as an entry point. 

To deploy a trained model, you'll use `<model>.deploy`, which takes in two arguments:
* **initial_instance_count**: The number of deployed instances (1).
* **instance_type**: The type of SageMaker instance for deployment.

Note: If you run into an instance error, it may be because you chose the wrong training or deployment instance_type. It may help to refer to your previous exercise code to see which types of instances we used.

In [51]:
%%time

# uncomment, if needed
# from sagemaker.pytorch import PyTorchModel


# deploy your model to create a predictor
predictor = model.deploy(initial_instance_count = 1,
                         instance_type = 'ml.t2.medium')


Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


-------------------!CPU times: user 344 ms, sys: 6.04 ms, total: 350 ms
Wall time: 9min 33s


In [98]:
## Pytorch
predictor = model.deploy(instance_type = 'ml.p2.xlarge',
                         initial_instance_count = 1)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


---------------------!

---
# Evaluating Your Model

Once your model is deployed, you can see how it performs when applied to our test data.

The provided cell below, reads in the test data, assuming it is stored locally in `data_dir` and named `test.csv`. The labels and features are extracted from the `.csv` file.

In [23]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import os

# read in test data, assuming it is stored locally
test_data = pd.read_csv(os.path.join(data_dir, "test.csv"), header=None, names=None)

# labels are in the first column
test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]

## EXERCISE: Determine the accuracy of your model

Use your deployed `predictor` to generate predicted, class labels for the test data. Compare those to the *true* labels, `test_y`, and calculate the accuracy as a value between 0 and 1.0 that indicates the fraction of test data that your model classified correctly. You may use [sklearn.metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) for this calculation.

**To pass this project, your model should get at least 90% test accuracy.**

In [90]:
test_x = test_x.astype('float32')

In [99]:
# First: generate predicted, class labels
test_y_preds = predictor.predict(test_x)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test that your model generates the correct number of labels
assert len(test_y_preds)==len(test_y), 'Unexpected number of predictions.'
print('Test passed!')

Test passed!


In [92]:
import numpy as np
def output_pytorch(preds):
    nor_preds = []
    nor_preds.append([1 if float(x) >= 0.5 else 0 for x in preds])
    np_preds = np.squeeze(np.array(nor_preds))
    return pd.Series(np_preds)

In [100]:
preds_pytorch = output_pytorch(test_y_preds)

In [79]:
preds_pytorch, type(preds_pytorch)

(0     0
 1     0
 2     0
 3     0
 4     0
 5     1
 6     0
 7     0
 8     0
 9     0
 10    0
 11    0
 12    0
 13    1
 14    1
 15    0
 16    0
 17    1
 18    0
 19    0
 20    0
 21    1
 22    1
 23    0
 24    0
 dtype: int64,
 pandas.core.series.Series)

In [80]:
preds_pytorch.shape, test_y.shape

((25,), (25,))

In [44]:
test_y_preds, type(test_y_preds)

(array([[0.49453649],
        [0.48884332],
        [0.48701373],
        [0.4820444 ],
        [0.4961589 ],
        [0.50274873],
        [0.48184648],
        [0.4710857 ],
        [0.47282401],
        [0.47515076],
        [0.47346422],
        [0.47582778],
        [0.47718614],
        [0.50347453],
        [0.50332206],
        [0.48278552],
        [0.48812303],
        [0.50234622],
        [0.47174525],
        [0.49651343],
        [0.47467998],
        [0.50100708],
        [0.50202954],
        [0.47097799],
        [0.47344574]]),
 numpy.ndarray)

In [46]:
test_y, type(test_y)

(0     1
 1     1
 2     1
 3     1
 4     1
 5     1
 6     0
 7     0
 8     0
 9     0
 10    0
 11    0
 12    1
 13    1
 14    1
 15    1
 16    1
 17    1
 18    0
 19    1
 20    0
 21    1
 22    1
 23    0
 24    0
 Name: 0, dtype: int64,
 pandas.core.series.Series)

In [47]:
# NNeighborsClassifer n = 5
from sklearn.metrics import accuracy_score
# Second: calculate the test accuracy
accuracy = accuracy_score(test_y, test_y_preds)

print(accuracy)


## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
print(test_y_preds)
print('\nTrue class labels: ')
print(test_y.values)

0.96

Predicted class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 0 1 0 1 1 0 0]

True class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]


In [53]:
# NNeighborsClassifer n = 3
from sklearn.metrics import accuracy_score
# Second: calculate the test accuracy
accuracy = accuracy_score(test_y, test_y_preds)

print(accuracy)


## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
print(test_y_preds)
print('\nTrue class labels: ')
print(test_y.values)

0.96

Predicted class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 0 1 0 1 1 0 0]

True class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]


In [85]:
# PyTorch epochs = 10
from sklearn.metrics import accuracy_score
# Second: calculate the test accuracy
accuracy = accuracy_score(test_y, preds_pytorch)

print(accuracy)


## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
print(preds_pytorch.values)
print('\nTrue class labels: ')
print(test_y.values)

0.64

Predicted class labels: 
[0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 1 1 0 0]

True class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]


In [93]:
# PyTorch epochs = 20
from sklearn.metrics import accuracy_score
# Second: calculate the test accuracy
accuracy = accuracy_score(test_y, preds_pytorch)

print(accuracy)


## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
print(preds_pytorch.values)
print('\nTrue class labels: ')
print(test_y.values)

0.64

Predicted class labels: 
[0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 1 1 0 0]

True class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]


In [101]:
# PyTorch epochs = 20, hidden_dim = 30
from sklearn.metrics import accuracy_score
# Second: calculate the test accuracy
accuracy = accuracy_score(test_y, preds_pytorch)

print(accuracy)


## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
print(preds_pytorch.values)
print('\nTrue class labels: ')
print(test_y.values)

0.6

Predicted class labels: 
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]

True class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]


### Question 1: How many false positives and false negatives did your model produce, if any? And why do you think this is?

** Answer**: 
For Scikit-learn model, I got 0 false positives and 1 false negatives. For Pytorch model, I got 10 false positives and 0 false negatives in PyTorch model with hidden_dim equals to 30. And I got 0 false positives and 9 negatives in PyTorch model with hidden_dim to 10. I think the reasons for two PyTorch models are overfitting in the first model and didn't learn well in the second model.

### Question 2: How did you decide on the type of model to use? 

** Answer**:

Because we got so small training data set, I think that is the most important impact for PyTorch model. Neural network could work well when we have a large data set. However, things go very well in scikit-learn's n neighbors classifer which performance higher than 95%. Therefore, I choose scikit-learn model to use. 

----
## EXERCISE: Clean up Resources

After you're done evaluating your model, **delete your model endpoint**. You can do this with a call to `.delete_endpoint()`. You need to show, in this notebook, that the endpoint was deleted. Any other resources, you may delete from the AWS console, and you will find more instructions on cleaning up all your resources, below.

In [102]:
# uncomment and fill in the line below!
# <name_of_deployed_predictor>.delete_endpoint()
predictor.delete_endpoint()

### Deleting S3 bucket

When you are *completely* done with training and testing models, you can also delete your entire S3 bucket. If you do this before you are done training your model, you'll have to recreate your S3 bucket and upload your training data again.

In [104]:
# deleting bucket, uncomment lines below

bucket_to_delete = boto3.resource('s3').Bucket(bucket)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': '599CED9ADC9A7560',
   'HostId': '+/y8bRaa4Vt6mEyy6fl1NU6xbnQkdi5e+G1xqpvQZuzmS3CvRqU3UtDLd4wonNNcZ1Zb+AA9QA8=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': '+/y8bRaa4Vt6mEyy6fl1NU6xbnQkdi5e+G1xqpvQZuzmS3CvRqU3UtDLd4wonNNcZ1Zb+AA9QA8=',
    'x-amz-request-id': '599CED9ADC9A7560',
    'date': 'Fri, 21 Aug 2020 03:36:47 GMT',
    'connection': 'close',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'sagemaker/moon-data/sagemaker-pytorch-2020-08-09-02-01-08-136/output/model.tar.gz'},
   {'Key': 'pytorch-training-2020-08-20-22-28-04-373/debug-output/index/000000000/000000000000_worker_0.json'},
   {'Key': 'sagemaker-scikit-learn-2020-08-19-06-43-31-360/source/sourcedir.tar.gz'},
   {'Key': 'counties/pca-2020-08-03-21-33-27-579/output/model.tar.gz'},
   {'Key': 'pytorch-training-2020-08-20-22-28-04-373/debug-output/claim.smd'},
   {'Key': 's

### Deleting all your models and instances

When you are _completely_ done with this project and do **not** ever want to revisit this notebook, you can choose to delete all of your SageMaker notebook instances and models by following [these instructions](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html). Before you delete this notebook instance, I recommend at least downloading a copy and saving it, locally.

---
## Further Directions

There are many ways to improve or add on to this project to expand your learning or make this more of a unique project for you. A few ideas are listed below:
* Train a classifier to predict the *category* (1-3) of plagiarism and not just plagiarized (1) or not (0).
* Utilize a different and larger dataset to see if this model can be extended to other types of plagiarism.
* Use language or character-level analysis to find different (and more) similarity features.
* Write a complete pipeline function that accepts a source text and submitted text file, and classifies the submitted text as plagiarized or not.
* Use API Gateway and a lambda function to deploy your model to a web application.

These are all just options for extending your work. If you've completed all the exercises in this notebook, you've completed a real-world application, and can proceed to submit your project. Great job!